<a href="https://colab.research.google.com/github/eriksali/DNN_2023_DL/blob/main/neo4j_protein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

# Load the graph data from kg_giant.csv
kg = pd.read_csv('kg_giant_protein.csv', low_memory=False)

# Define Neo4j connection details
uri = "neo4j+s://05b06d0e.databases.neo4j.io"
username = "neo4j"
password = "tsSmaXO6MCkspVXpU5BRIEnwCX7ooGDB4WWyQiageok"

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))


# Function to run Cypher queries
def run_query(query, params=None):
    with driver.session() as session:
        if params:
            result = session.run(query, parameters=params)
        else:
            result = session.run(query)
        return result

# Function to create nodes
def create_nodes():
    for idx, row in kg.iterrows():
        query = (
            f"MERGE (n:{row['x_type']} {{name: $x_name, source: $x_source}}) "
            f"MERGE (m:{row['y_type']} {{name: $y_name, source: $y_source}})"
        )
        params = {
            "x_name": row['x_name'],
            "x_source": row['x_source'],
            "y_name": row['y_name'],
            "y_source": row['y_source']
        }
        run_query(query, params=params)

# Function to create relationships
def create_relationships():
    for idx, row in kg.iterrows():
        query = (
            f"MATCH (n:{row['x_type']} {{name: $x_name, source: $x_source}}), "
            f"(m:{row['y_type']} {{name: $y_name, source: $y_source}}) "
            f"MERGE (n)-[:{row['relation']} {{display_relation: $display_relation}}]->(m)"
        )
        params = {
            "x_name": row['x_name'],
            "x_source": row['x_source'],
            "y_name": row['y_name'],
            "y_source": row['y_source'],
            "display_relation": row['display_relation']
        }
        run_query(query, params=params)

# Create nodes in Neo4j
create_nodes()

# Create relationships in Neo4j
create_relationships()

# Close the Neo4j driver
driver.close()
